In [1]:
!pip install -U accelerate>=0.21.0
!pip install torch -U
!pip install transformers -U
!pip install datasets spacy tqdm
!pip install seqeval
!pip install evaluate
!pip install pandas seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 7.0 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.2.0
    Uninstalling triton-2.2.0:
      Successfully uninstalled triton-2.2.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.19.3
    Uninstalling nvidia-nccl-cu12-2.19.3:
      Successfully uninstalled nvidia-nccl-cu12-2.19.3
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1+cu121
    Uninstalling torch-2.2.1+cu121:
      Successfully uninstalled torch-2.2.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2

In [2]:
import torch
# import torchtext

SEED = 1234
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

print("PyTorch Version: ", torch.__version__)
# print("torchtext Version: ", torchtext.__version__)
print(f"Using {'GPU' if str(DEVICE) == 'cuda' else 'CPU'}.")

PyTorch Version:  2.3.0+cu121
Using GPU.


In [15]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from datasets import load_dataset, load_dataset_builder, get_dataset_split_names, ClassLabel, Sequence

#description
dataset_name = "surrey-nlp/PLOD-CW"
ds_builder = load_dataset_builder(dataset_name)
print(ds_builder.info.description)
print(ds_builder.info.features)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'pos_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}


In [5]:
ds = load_dataset(dataset_name)

Generating train split:   0%|          | 0/1072 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/126 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/153 [00:00<?, ? examples/s]

In [6]:
from datasets import ClassLabel, Value, Sequence, Features

# Assuming `dataset` is your original dataset
def convert_format(example):
    # example[''] = str(example['id'])  # Convert id to string
    example['pos_tags'] = [ClassLabel(names=['ADJ', 'ADP', 'ADV', 'AUX', 'CONJ', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X', 'SPACE']).str2int(tag) for tag in example['pos_tags']]
    example['ner_tags'] = [ClassLabel(names=['B-O', 'B-AC', 'B-LF', 'I-LF']).str2int(tag) for tag in example['ner_tags']]
    return example

In [7]:
# Define the new features
new_features = Features({
    # 'id': Value(dtype='string', id=None),
    'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
    'pos_tags': Sequence(feature=ClassLabel(names=['ADJ', 'ADP', 'ADV', 'AUX', 'CONJ', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X', 'SPACE']), length=-1, id=None),
    'ner_tags': Sequence(feature=ClassLabel(names=['B-O', 'B-AC', 'B-LF', 'I-LF']), length=-1, id=None)
})

new_dataset = ds.map(convert_format, features=new_features)

Map:   0%|          | 0/1072 [00:00<?, ? examples/s]

Map:   0%|          | 0/126 [00:00<?, ? examples/s]

Map:   0%|          | 0/153 [00:00<?, ? examples/s]

In [8]:
label_list = new_dataset["train"].features[f"ner_tags"].feature.names

In [9]:
label_all_tokens = True
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True) ## For some models, you may need to set max_length to approximately 500.

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [17]:
import evaluate
import numpy as np
metric = evaluate.load("seqeval")
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }


In [13]:
from transformers import TrainingArguments, AutoConfig, AutoModelForTokenClassification, DataCollatorForTokenClassification, AutoTokenizer, Trainer
import accelerate
new_dataset = ds.map(convert_format, features=new_features)

task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "bert-base-cased"
batch_size = 4
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# tokenized_datasets = new_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_datasets = new_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=new_dataset["train"].column_names,
)
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer) #pads all of them to be of the same size
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {v: k for k, v in id2label.items()}
print("id2label", id2label)
print("label2id", label2id)

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)


args = TrainingArguments(
    "bert-finetuned-ner-for-deployment",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    push_to_hub=True,
)

Map:   0%|          | 0/126 [00:00<?, ? examples/s]

id2label {0: 'B-O', 1: 'B-AC', 2: 'B-LF', 3: 'I-LF'}
label2id {'B-O': 0, 'B-AC': 1, 'B-LF': 2, 'I-LF': 3}


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.232356,0.930222,0.932843,0.931530,0.926611
2,No log,0.244610,0.936541,0.926425,0.931455,0.926466
3,No log,0.273757,0.930361,0.924389,0.927366,0.923144
4,0.150100,0.287683,0.931554,0.928929,0.930240,0.924588
5,0.150100,0.295480,0.930251,0.929086,0.929668,0.924299


TrainOutput(global_step=670, training_loss=0.13016880377015072, metrics={'train_runtime': 185.2547, 'train_samples_per_second': 28.933, 'train_steps_per_second': 3.617, 'total_flos': 303358771482048.0, 'train_loss': 0.13016880377015072, 'epoch': 5.0})

In [ ]:
text = ds["test"][6]["tokens"]
labels = ds["test"][6]["ner_tags"]
model.to(DEVICE)
inputs = tokenizer(text, return_tensors="pt",  truncation=True, is_split_into_words=True, return_offsets_mapping=True)
inputs = {key: value.to(DEVICE) for key, value in inputs.items()}
offset_mapping = inputs['offset_mapping'].cpu().tolist()[0]
del inputs["offset_mapping"]

predictions = 0
# Example forward pass
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    # Move the tensor to the CPU if it's on the GPU
    pre_predictions = predictions
    predictions = predictions.cpu().view(-1).tolist()

input_ids = inputs['input_ids'].cpu().view(-1).tolist()
attention_mask = inputs['attention_mask'].cpu().view(-1).tolist()

aggregated_predictions = []
for i, offset in enumerate(offset_mapping):
  # Ignore special tokens
  if sum(offset) == 0:
    continue
  # If the offset's start position is 0, it's a new word
  if offset[0] == 0:
    aggregated_predictions.append(predictions[i])


print(len(aggregated_predictions))

In [ ]:


predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)


#prediction was what model predicted and labels is what is actually is.
# Remove the predictions for the [CLS] and [SEP] tokens
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

# Compute multiple metrics on the test restuls
results = metric.compute(predictions=true_predictions, references=true_labels)
results